# Binary Integer Programming

$x_{ij}=\begin{cases}1\\ 0\end{cases}$ if coin i is assigned to child j, otherwise.

The integer linear programming formulation of this problem becomes

maximize $z=\sum_{i=1}^{N}\sum_{j=1}^{M}x_{ij}$

subject to:

$\sum_{i=1}^{N}c_{i}x_{ij}=\frac{1}{M}\sum_{i=1}^{N}c_{i} \quad j=1,2,...,M$

$\sum_{j=1}^{M}x_{ij}=1 \quad i=1,2,...,N$

$x_{ij}\in\{0,1\} \quad i=1,2,...,N; \quad j=1,2,...,M$

In [7]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

In [8]:
def mixed_integer_linear_programming(direction, A, senses, b, c, l, u, types):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist(), types = types.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(), col_indices.tolist(), A.data.tolist()))

    print(prob.write_as_string())
    # solve the problem
    prob.solve()
    
    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)

In [9]:
def coin_distribution_problem(coins_file, M):
    # your implementation starts below
    coin = np.loadtxt(coins_file)
    N = coin.size
    
    l= np.repeat(0, N*M)
    u= np.repeat(1, N*M)
    c= np.repeat(1, N*M)
    b= np.concatenate(( np.repeat(np.sum(coin)/M, M), np.repeat(1,N))).astype(int)
    senses= np.repeat("E", M+N)
    types = np.repeat("B", M*N)
    
    aij= np.concatenate(( np.repeat(coin.astype(int), M), np.repeat(1,M*N)))
    row = np.concatenate(( np.tile(range(M), N), np.repeat(np.arange(M, M+N), M)))
    col = np.tile(range(M*N), 2)
    A = sp.csr_matrix((aij, (row, col)), shape = (M+N,M*N))
    
    x_star, obj_star = mixed_integer_linear_programming("minimize", A, senses, b, c, l, u, types)
    X_star = np.array(x_star).reshape(N,M)
    # your implementation ends above
    return(X_star)

In [10]:
X_star = coin_distribution_problem("coins.txt", 2)
print(X_star)

Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Minimize
 obj1: x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8
Subject To
 c1: 10 x1 + 5 x3 + 25 x5 + 10 x7  = 25
 c2: 10 x2 + 5 x4 + 25 x6 + 10 x8  = 25
 c3: x1 + x2  = 1
 c4: x3 + x4  = 1
 c5: x5 + x6  = 1
 c6: x7 + x8  = 1
Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
Binaries
 x1  x2  x3  x4  x5  x6  x7  x8 
End

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 4.000000 after 0.01 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 3 rows and 1 columns.
MIP Presolve modified 2 coefficients.
Aggregator did 7 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.0